# 인공지능과 기계학습 12주차 과제

신용카드 연체 여부(`default.payment.next.month`)를 예측하는 머신러닝 모델을 구축

## 과제 내용
- 데이터 로드 및 전처리 (`ID`, 범주형 변수 제거)
- 데이터 분할 (8:2)
- 정규화 (`StandardScaler`)
- 차원 축소 (`PCA`로 90% 이상 설명 주성분)
- 분류 모델 학습 (`LinearSVC`, class_weight='balanced') + 교차검증
- 최종 테스트 평가 (`classification_report`)


## 1. 데이터 로드 & 전처리

In [1]:
# 데이터 로드

import pandas as pd
df = pd.read_csv('/Users/dankim/AIML/UCI_Credit_Card.csv')

In [2]:
# 데이터 전처리

# ID 컬럼 제거
df = df.drop(columns=['ID'])

# 범주형 변수 제거
df = df.drop(columns=['SEX', 'EDUCATION', 'MARRIAGE'])

# 특징과 타겟 분리
X = df.drop(columns=['default.payment.next.month'])
y = df['default.payment.next.month']

print(X.shape)
print(y.value_counts())

(30000, 20)
default.payment.next.month
0    23364
1     6636
Name: count, dtype: int64


## 2. 데이터 분할

In [3]:
# 데이터 분할

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42 # train_test_split을 이용해 8:2로 학습 및 검증 데이터 분할, random_state = 42 고정
)

## 3. 스케일링

In [4]:
# 스케일링

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # StandardScaler를 사용하여 정규화
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 4. 차원 축소

In [5]:
# 차원 축소
# PCA를 이용해 전체 분산의 90% 이상을 설명하는 최소한의 주성분 수를 자동으로 선택

from sklearn.decomposition import PCA
pca = PCA(n_components=0.90)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
print(f'선택된 최소한의 주성분 수: {pca.n_components_}')

선택된 최소한의 주성분 수: 10


## 5. 분류 모델 학습

In [6]:
# 모델 학습 및 5-fold 교차검증

from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

model = LinearSVC(class_weight='balanced', max_iter=10000) # 분류기 클래스에 class_weight='balanced' 추가 
# (해당 옵션은 불균형한 데이터에서 소수 클래스도 잘 학습하도록 돕는 옵션)
scores = cross_val_score(model, X_train_pca, y_train, cv=5, scoring='accuracy')

print(f'5-fold 교차 검증 평균 정확도: {scores.mean():.4f}')

5-fold 교차 검증 평균 정확도: 0.6805


## 6. 최종 평가

In [7]:
# 최종 성능 평가
from sklearn.metrics import classification_report

model.fit(X_train_pca, y_train)
y_pred = model.predict(X_test_pca)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.70      0.78      4687
           1       0.37      0.62      0.46      1313

    accuracy                           0.68      6000
   macro avg       0.62      0.66      0.62      6000
weighted avg       0.76      0.68      0.71      6000

